# Imports et pathways

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
#---
# On a besoin du path vers tous les DFs
#---
path_SJ="../../../SAJESS/"
#---
# Puis vers les différents dossiers du disque
#---
path_Obs="SJ_Observations/"
path_Parsivel="SJ_Parsivel_Data/"
path_Met="SJ_Met_Tripod/"
path_ECCC="SJ_ECCC_Edmunston_1min/"

# Identification des timeframes où on a des tempêtes de neiges

In [3]:
#---
# Dans les observations, on utilise le fichier "detailed_obs_summary.txt",
# qui donne les débuts et fin des tempêtes, avec les types de précipitations principaux observés.
#---

Obs_Summary = pd.read_csv(path_SJ+path_Obs+"detailed_obs_summary.txt")
Obs_Summary.start=pd.to_datetime(Obs_Summary['start'], format='mixed')
Obs_Summary.end=pd.to_datetime(Obs_Summary['end'], format='mixed')

#Obs_Summary

In [4]:
#---
# On trouve tous les moments avec des tempêtes principalement de neige. 
# On trouve 11 tempêtes:
#---
Obs_Summary_gr = Obs_Summary.groupby("type").get_group("SN")
Obs_Summary_gr=Obs_Summary_gr.reset_index()

del Obs_Summary_gr["index"]

Obs_Summary_gr
#---
# "Obs_Summary_gr" donne le début et la fin de chaque tempête de neige en mars et en avril
# Les tempêtes 0 à 5 sont en mars
# Les tempêtes 6 à 10 sont en avril
#---

,start,end,type
0,2021-03-01 11:00:00,2021-03-02 00:00:00,SN
1,2021-03-09 11:10:00,2021-03-09 20:40:00,SN
2,2021-03-13 05:40:00,2021-03-13 08:50:00,SN
3,2021-03-14 10:00:00,2021-03-14 12:40:00,SN
4,2021-03-27 02:10:00,2021-03-27 09:20:00,SN
5,2021-03-28 12:10:00,2021-03-29 19:30:00,SN
6,2021-04-01 11:10:00,2021-04-02 00:50:00,SN
7,2021-04-02 15:30:00,2021-04-02 20:50:00,SN
8,2021-04-04 20:20:00,2021-04-05 13:50:00,SN
9,2021-04-17 12:20:00,2021-04-17 13:50:00,SN


In [5]:
Obs_Summary_gr["Temp_min"]=np.nan
Obs_Summary_gr["Temp_max"]=np.nan
Obs_Summary_gr["Temp_mean"]=np.nan
Obs_Summary_gr

,start,end,type,Temp_min,Temp_max,Temp_mean
0,2021-03-01 11:00:00,2021-03-02 00:00:00,SN,NaN,NaN,NaN
1,2021-03-09 11:10:00,2021-03-09 20:40:00,SN,NaN,NaN,NaN
2,2021-03-13 05:40:00,2021-03-13 08:50:00,SN,NaN,NaN,NaN
3,2021-03-14 10:00:00,2021-03-14 12:40:00,SN,NaN,NaN,NaN
4,2021-03-27 02:10:00,2021-03-27 09:20:00,SN,NaN,NaN,NaN
5,2021-03-28 12:10:00,2021-03-29 19:30:00,SN,NaN,NaN,NaN
6,2021-04-01 11:10:00,2021-04-02 00:50:00,SN,NaN,NaN,NaN
7,2021-04-02 15:30:00,2021-04-02 20:50:00,SN,NaN,NaN,NaN
8,2021-04-04 20:20:00,2021-04-05 13:50:00,SN,NaN,NaN,NaN
9,2021-04-17 12:20:00,2021-04-17 13:50:00,SN,NaN,NaN,NaN


In [6]:
#---
# fonction pour simplifier l'accès au start et end des tempêtes en fonction de leur indice
#---
def storm_tf(index):
    start=Obs_Summary_gr["start"][index]
    end=Obs_Summary_gr["end"][index]
    return (start,end)

# Identification des températures moyennes dans les tempêtes de neige
## (On cherche celles sous 0°C)

Pour chaque tempête, en mars et en avril, je vais chercher la Tmoy, Tmin et Tmax, pour voir lesquelles sont globalement sous 0°C. <br> Pour ce faire, je vais utiliser les données du Met Tripod utilisé pendant la campagne SAJESS

## MARS

In [7]:
#---
# Dans les observations, on utilise le fichier "detailed_obs_summary.txt",
# qui donne les débuts et fin des tempêtes, avec les types de précipitations principaux observés.
#---
Met_Tripod = pd.read_csv(path_SJ+path_Met+"SJ_MT_MET_01_MAS_1min_202103.txt")
Met_Tripod.set_index("Timestamp", inplace=True)

Met_Tripod.index = pd.to_datetime(Met_Tripod.index)

Met_Tripod

,RECORD,Air_Temp,Humidity,Dew_Point,Vapour_press_kPa,SW_Downwelling,SW_Upwelling,LW_Downwelling,LW_Upwelling,SW_Net,LW_Net,Albedo,Net_Radiation,IR_1_Temp,IR_2_Temp,Soil_WC,Soil_EC,Soil_Temp,SR50A_SnowDepth,SR50A_QualityVal
Timestamp,,,,,,,,,,,,,,,,,,,,
2021-03-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-01 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-01 00:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-01 00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-01 00:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-31 23:55:00,40279.0,10.77034,75.91,6.729018,0.982380,1.277408,0.663479,347.0450,339.3288,0.613930,7.716245,0.519328,8.330174,3.880300,5.688875,NaN,NaN,NaN,-1.391586,168.0
2021-03-31 23:56:00,40280.0,10.76974,75.72,6.627844,0.975570,0.977021,0.294839,347.2334,338.8698,0.682183,8.363655,0.295314,9.045837,3.828183,5.531583,NaN,NaN,NaN,-1.392737,169.0
2021-03-31 23:57:00,40281.0,10.75570,75.51,6.661291,0.977822,0.845062,0.074894,347.1096,338.5287,0.770168,8.580899,0.067194,9.351068,3.775817,5.517433,NaN,NaN,NaN,-1.391759,169.0


In [8]:
#---
# On extrait les données en dfs différents pour chaque tempêtes de mars
#---
for n in range(6):
    Met_record = Met_Tripod.loc[storm_tf(n)[0] : storm_tf(n)[1]]
    Met_record.to_csv(f"../../codes_export/dataframe/ETE24/Met_2021_03_storm_{n}.csv",index=True)


In [9]:
#---
# On extrait les Tmin, Tmax et Tmoy de chaque tempête, en extrayant la colonne des températures comme liste.
# On ajoute ces informations aux tableau des tempêtes, pour voir lesquelles utiliser.
#---
#---
# MARS
#---
for n in range(6):

    #On va chercher directement la colonne des températures
    Met_T_list = Met_Tripod.loc[storm_tf(n)[0] : storm_tf(n)[1]]["Air_Temp"]
    #On trouve les min, max et moy
    #print(Met_T_list.min(),Met_T_list.max(),Met_T_list.mean())
    Obs_Summary_gr.at[n,"Temp_min"]=Met_T_list.min()
    Obs_Summary_gr.at[n,"Temp_max"]=Met_T_list.max()
    Obs_Summary_gr.at[n,"Temp_mean"]=Met_T_list.mean()
 
Obs_Summary_gr


,start,end,type,Temp_min,Temp_max,Temp_mean
0,2021-03-01 11:00:00,2021-03-02 00:00:00,SN,NaN,NaN,NaN
1,2021-03-09 11:10:00,2021-03-09 20:40:00,SN,-16.153020,-2.235767,-8.117721
2,2021-03-13 05:40:00,2021-03-13 08:50:00,SN,-3.861495,-1.357761,-2.579345
3,2021-03-14 10:00:00,2021-03-14 12:40:00,SN,-10.375430,-9.007304,-10.036219
4,2021-03-27 02:10:00,2021-03-27 09:20:00,SN,-2.779660,1.526459,-0.703188
5,2021-03-28 12:10:00,2021-03-29 19:30:00,SN,-4.692759,2.765668,-0.733800
6,2021-04-01 11:10:00,2021-04-02 00:50:00,SN,NaN,NaN,NaN
7,2021-04-02 15:30:00,2021-04-02 20:50:00,SN,NaN,NaN,NaN
8,2021-04-04 20:20:00,2021-04-05 13:50:00,SN,NaN,NaN,NaN
9,2021-04-17 12:20:00,2021-04-17 13:50:00,SN,NaN,NaN,NaN


Les tempêtes de mars semblent toutes convenir au choix du treshold de température choisit, vu les températures min et moyennes. Il y a 2 tempêtes qui montent à plus de 0°C, on pourra éliminer les données au-dessus de ces températures au besoin.

## AVRIL

In [10]:
#---
# Dans les observations, on utilise le fichier "detailed_obs_summary.txt",
# qui donne les débuts et fin des tempêtes, avec les types de précipitations principaux observés.
#---
Met_Tripod = pd.read_csv(path_SJ+path_Met+"SJ_MT_MET_01_MAS_1min_202104.txt")
Met_Tripod.set_index("Timestamp", inplace=True)

Met_Tripod.index = pd.to_datetime(Met_Tripod.index)

Met_Tripod

,RECORD,Air_Temp,Humidity,Dew_Point,Vapour_press_kPa,SW_Downwelling,SW_Upwelling,LW_Downwelling,LW_Upwelling,SW_Net,LW_Net,Albedo,Net_Radiation,IR_1_Temp,IR_2_Temp,Soil_WC,Soil_EC,Soil_Temp,SR50A_SnowDepth,SR50A_QualityVal
Timestamp,,,,,,,,,,,,,,,,,,,,
2021-04-01 00:00:00,40284.0,10.51098,77.66,6.669386,0.978383,0.823685,0.068134,347.7823,340.2276,0.755550,7.554703,-0.083081,8.310252,4.084066,5.843174,NaN,NaN,NaN,-1.390952,169.0
2021-04-01 00:01:00,40285.0,10.53468,76.66,6.681806,0.979198,1.177274,0.613499,347.6405,339.6025,0.563776,8.037990,0.518422,8.601765,3.933708,5.687216,NaN,NaN,NaN,-1.393089,169.0
2021-04-01 00:02:00,40286.0,10.61689,77.13,6.722259,0.981928,0.993625,0.325879,347.9081,339.5221,0.667745,8.385910,0.325818,9.053655,3.992550,5.706183,NaN,NaN,NaN,-1.392539,168.0
2021-04-01 00:03:00,40287.0,10.58902,76.80,6.744144,0.983400,1.219466,0.694978,347.8412,339.8325,0.524488,8.008636,0.568793,8.533125,4.018750,5.762775,NaN,NaN,NaN,-1.392164,169.0
2021-04-01 00:04:00,40288.0,10.58011,76.42,6.662653,0.977910,1.085045,0.641609,347.6093,339.8891,0.443436,7.720202,0.589114,8.163637,4.057900,5.785700,NaN,NaN,NaN,-1.392276,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-30 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-30 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-30 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#---
# AVRIL
#---
for n in range(5):
   
    #On va chercher directement la colonne des températures
    Met_T_list = Met_Tripod.loc[storm_tf(n+6)[0] : storm_tf(n+6)[1]]["Air_Temp"]
    #On trouve les min, max et moy
    #print(Met_T_list.min(),Met_T_list.max(),Met_T_list.mean())
    Obs_Summary_gr.at[n+6,"Temp_min"]=Met_T_list.min()
    Obs_Summary_gr.at[n+6,"Temp_max"]=Met_T_list.max()
    Obs_Summary_gr.at[n+6,"Temp_mean"]=Met_T_list.mean()
 
Obs_Summary_gr

,start,end,type,Temp_min,Temp_max,Temp_mean
0,2021-03-01 11:00:00,2021-03-02 00:00:00,SN,NaN,NaN,NaN
1,2021-03-09 11:10:00,2021-03-09 20:40:00,SN,-16.153020,-2.235767,-8.117721
2,2021-03-13 05:40:00,2021-03-13 08:50:00,SN,-3.861495,-1.357761,-2.579345
3,2021-03-14 10:00:00,2021-03-14 12:40:00,SN,-10.375430,-9.007304,-10.036219
4,2021-03-27 02:10:00,2021-03-27 09:20:00,SN,-2.779660,1.526459,-0.703188
5,2021-03-28 12:10:00,2021-03-29 19:30:00,SN,-4.692759,2.765668,-0.733800
6,2021-04-01 11:10:00,2021-04-02 00:50:00,SN,-3.727957,0.831890,-0.804353
7,2021-04-02 15:30:00,2021-04-02 20:50:00,SN,-4.316516,-3.325047,-3.699031
8,2021-04-04 20:20:00,2021-04-05 13:50:00,SN,-1.118470,1.953411,0.403815
9,2021-04-17 12:20:00,2021-04-17 13:50:00,SN,1.523077,3.667377,2.409237


Pour mars et avril, on va utiliser les tempêtes 1, 2, 3, 4, 5, 6, 7 et 10. Les tempêtes 0, 8 et 9 sont trop chaudes en moyennes, ou il n'y a pas de données suffisantes.

On doit maintenant calculer le nombre de photos prises dans chaques tempêtes.

In [12]:
Obs_Summary_gr = Obs_Summary_gr.drop([0,8,9])
Obs_Summary_gr
Obs_Summary_gr.to_csv(f"../../codes_export/dataframe/ETE24/Snowstorms_For_MASC_Validation.csv",index=True)

In [13]:
#---
# On extrait les données en dfs différents pour chaque tempêtes de avril
#---
for n in [6,7,10]:
    Met_record = Met_Tripod.loc[storm_tf(n)[0] : storm_tf(n)[1]]
    Met_record.to_csv(f"../../codes_export/dataframe/ETE24/Met_2021_04_storm_{n}.csv",index=True)